# Code to Load the Final Model from Hugging Face and Skip Training Steps

In [ ]:
'''
!pip install unsloth accelerate transformers datasets gradio

from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    "ras6899/symptom-sft-dpo",
    max_seq_length=2048,
    load_in_4bit=True,           # Efficient inference
)

FastLanguageModel.for_inference(model)
model.eval()

print("Model loaded successfully!")
'''


# Setup and Base Model

In [ ]:
%%capture
# Import basic utilities
import os, re, pprint

# Check if we're in Google Colab and install appropriate packages
if "COLAB_" not in "".join(os.environ.keys()):
    # For non-Colab environments, a simple install is sufficient
    !pip install unsloth
else:
    # For Colab, we need more specific installations due to environment constraints
    # Get the torch version to ensure compatible installations
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")

    # Install dependencies without overwriting existing packages
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth

# Install specific versions of transformers and trl for compatibility
!pip install transformers==4.55.4
!pip install --no-deps trl==0.22.2

# Technique 1: Supervised Fine-Tuning (SFT)

## Load Model and LoRA Parameters

In [ ]:
from unsloth import FastLanguageModel
import torch

# Load Model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-3B-Instruct",
    max_seq_length=2048,
    dtype=torch.bfloat16,                          # REQUIRED for DPO
    load_in_4bit=False,                            # MUST be full precision for DPO
)

# Add LoRA Parameters
model = FastLanguageModel.get_peft_model(
    model,
    r=8,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
)


==((====))==  Unsloth 2025.12.4: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.12.4 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


## Load SFT Dataset for Training

In [ ]:
import pandas as pd
from datasets import Dataset

# Load structured dataset
df = pd.read_csv("dataset_sft.csv")

# Build training example
def build_example(example):
    symptoms = example["Input"].strip()
    output = example["Output"].strip()

    prompt = (
        f"Symptoms: {symptoms}\n"
        f"Classify the symptoms using ONLY one of these conditions: "
        f"Common Cold, Flu, Covid-19, Bronchitis, Allergies, Strep Throat, Stomach Flu, Unknown. "
        f"Choose Unknown when symptoms are vague, unclear, contradictory, or do not match the listed conditions.\n"
        f"Condition and Guidance:\n"
        f"{output}"
    )
    return {"text": prompt}


# Load dataset
dataset = Dataset.from_pandas(df)

# Map — Input/Output MUST exist here
dataset = dataset.map(build_example)

# Remove old columns AFTER mapping
dataset = dataset.remove_columns([col for col in dataset.column_names if col != "text"])

# Confirm what's inside (VERY IMPORTANT)
print(dataset[0]["text"])
print("----")
print(dataset[1]["text"])
print("----")
print(dataset[2]["text"])


Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Symptoms: Sneezing, runny nose, mild sore throat, light cough, low energy.
Classify the symptoms using ONLY one of these conditions: Common Cold, Flu, Covid-19, Bronchitis, Allergies, Strep Throat, Stomach Flu, Unknown. Choose Unknown when symptoms are vague, unclear, contradictory, or do not match the listed conditions.
Condition and Guidance:
Condition: Common Cold
Guidance: The symptoms are commonly associated with common cold. Next steps are to rest, stay hypdrated, and monitor your symptoms. Consider taking over-the-counter medications. Safety note: This is not a diagnosis. Please consult a healthcare professional if symptoms worsen or persist.
----
Symptoms: Stuffy nose, mild headache, scratchy throat, no fever.
Classify the symptoms using ONLY one of these conditions: Common Cold, Flu, Covid-19, Bronchitis, Allergies, Strep Throat, Stomach Flu, Unknown. Choose Unknown when symptoms are vague, unclear, contradictory, or do not match the listed conditions.
Condition and Guidance:


## Train Model

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Train with SFT
training_args = TrainingArguments(
    output_dir="./model-medical-sft",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=50,
    max_steps=300,
    learning_rate=2e-4,
    bf16=True,
    fp16=False,
    logging_steps=25,
    save_steps=250,
    report_to="none",   # <-- disable W&B, MLFlow, all reporters
)


trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    args = training_args,
    max_seq_length = 2048,
)

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/90 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 90 | Num Epochs = 25 | Total steps = 300
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 4,587,520 of 3,217,337,344 (0.14% trained)


Step,Training Loss
25,2.369800
50,0.834100
75,0.295900
100,0.213400
125,0.182400
150,0.154100
175,0.125900
200,0.103100
225,0.066300
250,0.044100


TrainOutput(global_step=300, training_loss=0.3725006874402364, metrics={'train_runtime': 293.7845, 'train_samples_per_second': 8.169, 'train_steps_per_second': 1.021, 'total_flos': 5218674838020096.0, 'train_loss': 0.3725006874402364, 'epoch': 25.0})

## Save SFT Model

In [ ]:
# ================================
# ⭐ SAVE TRAINED MODEL (LoRA)
# ================================
save_path = "./model-medical-sft-final"

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model saved to: {save_path}")


Model saved to: ./model-medical-sft-final


## SFT Evaluation

In [ ]:
# ================================
# ⭐ Inference using your trained model
# ================================
from unsloth import FastLanguageModel
import torch

# Load trained model (LoRA merged)
model, tokenizer = FastLanguageModel.from_pretrained(
    "./model-medical-sft-final",
    max_seq_length=2048,
    load_in_4bit=True,
)
FastLanguageModel.for_inference(model)
model.eval()

# ================================
# ⭐ Generate Condition + Guidance
# ================================
def generate_condition_and_guidance(symptoms):
    prompt = (
        f"Symptoms: {symptoms}\n"
        f"Classify the symptoms using ONLY one of these conditions: "
        f"Common Cold, Flu, Covid-19, Bronchitis, Allergies, Strep Throat, Stomach Flu, Unknown. "
        f"Choose Unknown when symptoms are vague, unclear, not medically specific, or do not match the listed conditions.\n"
        f"Condition and Guidance:\n"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(next(model.parameters()).device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=150,
            do_sample=False,  # Deterministic
            temperature=0.0,
        )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)

    # Extract only the model continuation
    if "Condition and Guidance:" in decoded:
        decoded = decoded.split("Condition and Guidance:", 1)[1].strip()

    return decoded

# ================================
# ⭐ Test your model
# ================================
test_symptoms = [
    "I have a high fever, chills, body aches, and a dry cough that started quickly.", # Happy Case 1
    "I suddenly lost my sense of taste and smell and have a mild cough.", # Happy Case 2
    "I have a very sore throat with white patches and pain when swallowing.", # Happy Case 3
    "I have itchy eyes, sneezing, and a runny nose but feel fine otherwise.", # Happy Case 4
    "I am having nausea, stomach cramps, diarrhea, and fatigue.", # Happy Case 5
    "I'm tired and off but maybe it's stress.", # Edge-Case 1
    "I feel funny but there's nothing wrong.", # Edge-Case 2
    "I have a weird bump on my neck that came out of no where. Could this be cancer?", # Edge-Case 3,
    "I have a high fever, persistent cough, itchy eyes, sneezing, and a sore throat that hurts when I swallow.", # Failure Case
    "Help!"
]

for s in test_symptoms:
    print("\n====================================")
    print("Symptoms:", s)
    print("\nModel Output:")
    print(generate_condition_and_guidance(s))


==((====))==  Unsloth 2025.12.4: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

Symptoms: I have a high fever, chills, body aches, and a dry cough that started quickly.

Model Output:
Condition: Flu
Guidance: The symptoms are commonly associated with flu. Next steps are to rest, drink plenty of fluids, and manage fever/body aches with OTC medications. High-risk individuals should consider contacting a provider regarding antivirals. Safety note: This is not a diagnosis. Please consult a healthcare professional if symptoms worsen or persist.

Symptoms: I suddenly lost my sense of taste and smell and have a mild c

# Transition Block

In [ ]:
# ============================================================
# 🔄 TRANSITION BLOCK: Reset Python runtime for clean DPO stage
# ============================================================

# IMPORTANT:
# Unsloth monkey-patches TRL's trainers,
# so we must restart the Python process BEFORE DPO training.

print("SFT complete. Saving SFT mxodel... Runtime will now restart for DPO training.")

import os
os.kill(os.getpid(), 9)   # <-- HARD RESTART (safe for Colab)


## Reload SFT Model

In [ ]:
# ================================
# ⭐ RELOAD TRAINED MODEL
# ================================
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    save_path,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,   # same as during training
)

FastLanguageModel.for_inference(model)
model.eval()

print("Model reloaded and ready for inference!")

# Technique 2: Direct Preference Optimization (DPO)

In [ ]:
!pip install unsloth accelerate transformers datasets trl

INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.6/766.6 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.2/253.2 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 133.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 105.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5

## Load SFT Model as Base

In [ ]:
from unsloth import FastLanguageModel
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

model, tokenizer = FastLanguageModel.from_pretrained(
    "./model-medical-sft-final",
    max_seq_length=2048,
    dtype=torch.bfloat16,
    load_in_4bit=False,
)

model = FastLanguageModel.for_training(model)
model.to(device)
tokenizer.pad_token = tokenizer.eos_token

print("Loaded SFT symptom model with LoRA.")


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


==((====))==  Unsloth 2025.12.4: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Unsloth 2025.12.4 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


Loaded SFT symptom model with LoRA.


## Load DPO Dataset for Training

In [ ]:
from datasets import load_dataset

# Load JSONL file as a single split
raw_dataset = load_dataset(
    "json",
    data_files="dataset_dpo.jsonl",
    split="train"
)

print(f"Total DPO examples: {len(raw_dataset)}")

# Create a small train/test split (e.g., 80/20)
dataset = raw_dataset.train_test_split(test_size=0.2, seed=42)
print(f"Train examples: {len(dataset['train'])}")
print(f"Test examples:  {len(dataset['test'])}")

# Inspect structure
example = dataset["train"][0]
print("\nExample keys:", example.keys())
print("\nSample example:")
print("PROMPT:", example["prompt"])
print("\nCHOSEN:", example["chosen"])
print("\nREJECTED:", example["rejected"])


Generating train split: 0 examples [00:00, ? examples/s]

Total DPO examples: 85
Train examples: 68
Test examples:  17

Example keys: dict_keys(['prompt', 'chosen', 'rejected'])

Sample example:
PROMPT: My nose keeps running.

CHOSEN: Condition: Cold
Guidance: A runny nose often comes from a mild cold. Rest, drink warm fluids, and give it time to improve.
Safety note: This is not a diagnosis. Please consult a healthcare professional if symptoms worsen or persist.

REJECTED: Condition: Nasal Drainage Disorder
Guidance: This indicates nasal drainage disorder. Here is an extended breakdown of mucus membrane function and treatment phases. This is for educational purposes only.
Safety note: This is not a diagnosis. Please consult a healthcare professional if symptoms worsen or persist.


In [ ]:
from multiprocessing import cpu_count

def apply_chat_template(example):
    chosen_messages = [
        {"role": "user", "content": example["prompt"]},
        {"role": "assistant", "content": example["chosen"]},
    ]
    rejected_messages = [
        {"role": "user", "content": example["prompt"]},
        {"role": "assistant", "content": example["rejected"]},
    ]

    example["chosen"] = tokenizer.apply_chat_template(
        chosen_messages,
        tokenize=False,
        add_generation_prompt=False,
    )
    example["rejected"] = tokenizer.apply_chat_template(
        rejected_messages,
        tokenize=False,
        add_generation_prompt=False,
    )

    return example

processed_dataset = dataset.map(
    apply_chat_template,
    num_proc=cpu_count(),
    desc="Formatting symptom DPO pairs with chat template",
)

print("Processed dataset splits:", processed_dataset.keys())
print("Processed train size:", len(processed_dataset["train"]))
print("Processed test size:", len(processed_dataset["test"]))


Formatting symptom DPO pairs with chat template (num_proc=12):   0%|          | 0/68 [00:00<?, ? examples/s]

Formatting symptom DPO pairs with chat template (num_proc=12):   0%|          | 0/17 [00:00<?, ? examples/s]

Processed dataset splits: dict_keys(['train', 'test'])
Processed train size: 68
Processed test size: 17


## Define symptom_inference

In [ ]:
import torch
from transformers import TextStreamer

# Assign Output Rules
def symptom_inference(prompt, max_new_tokens=180):
    system_prompt = (
        "You classify symptoms into exactly one of the following conditions: "
        "cold, flu, COVID-19, allergies, strep throat, stomach flu, bronchitis, or Unknown. "
        "If symptoms do not clearly fit one condition, choose Unknown.\n\n"
        "Format:\n"
        "Condition: <one condition>\n"
        "Guidance: <one short, calm paragraph with 2–3 gentle suggestions>\n"
        "Safety note: This is not a diagnosis. Please consult a healthcare professional if symptoms worsen or persist.\n"
        "Keep the tone warm, simple, and concise. Do not invent conditions."
    )

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
    ]

    formatted = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

    encoded = tokenizer(formatted, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(
            **encoded,
            max_new_tokens=max_new_tokens,
            temperature=0.15,
            top_p=0.92,
            repetition_penalty=1.1,
        )

    full_text = tokenizer.decode(outputs[0], skip_special_tokens=False)

    # Extract assistant message
    if "<|start_header_id|>assistant<|end_header_id|>" in full_text:
        out = full_text.split("<|start_header_id|>assistant<|end_header_id|>")[-1]
    else:
        out = full_text.split("assistant")[-1]

    return out.replace("<|eot_id|>", "").strip()



In [ ]:
from unsloth import PatchDPOTrainer

PatchDPOTrainer()  # <-- important: patches TRL's DPOTrainer for Unsloth

## Set Hyperparameters

In [ ]:
# Set Hyperparameters
from trl import DPOTrainer, DPOConfig
import datetime

gradient_accumulation_steps = 8
warmup_ratio = 0.1
learning_rate = 1e-6
beta = 0.1            # DPO tradeoff parameter
weight_decay = 0.0

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = f"outputs/symptom_dpo_{timestamp}"


## Create Trainer and Run Training

In [ ]:
# Create DPO Trainer
dpo_trainer = DPOTrainer(
    model=model,               # SFT + LoRA model
    ref_model=None,            # when None, patched trainer clones a frozen ref model
    args=DPOConfig(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=gradient_accumulation_steps,

        warmup_ratio=warmup_ratio,
        learning_rate=learning_rate,

        beta=beta,

        num_train_epochs=1,              # small dataset = 1–3 epochs

        optim="adamw_8bit",
        weight_decay=weight_decay,
        lr_scheduler_type="linear",

        seed=42,
        output_dir=output_dir,
        report_to="none",
        logging_steps=10,
        save_steps=50,

        eval_strategy="steps",
        eval_steps=20,
    ),
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["test"],

    tokenizer=tokenizer,
    max_length=1024,
    max_prompt_length=512,
)


Extracting prompt in train dataset (num_proc=16):   0%|          | 0/68 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=16):   0%|          | 0/68 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=16):   0%|          | 0/68 [00:00<?, ? examples/s]

Extracting prompt in eval dataset (num_proc=16):   0%|          | 0/17 [00:00<?, ? examples/s]

Applying chat template to eval dataset (num_proc=16):   0%|          | 0/17 [00:00<?, ? examples/s]

Tokenizing eval dataset (num_proc=16):   0%|          | 0/17 [00:00<?, ? examples/s]

In [ ]:
# Run DPO Training
print(f"Starting DPO training:")
print(f"- Training examples: {len(processed_dataset['train'])}")
print(f"- Evaluation examples: {len(processed_dataset['test'])}")
print(f"- Model: {model.config._name_or_path}")
print(f"- Output directory: {output_dir}")
print(f"- Beta value: {dpo_trainer.beta}")
print(f"- Training epochs: {dpo_trainer.args.num_train_epochs}")
print(f"- Learning rate: {dpo_trainer.args.learning_rate}")
print(f"- Batch size: {dpo_trainer.args.per_device_train_batch_size} * "
      f"{dpo_trainer.args.gradient_accumulation_steps} (grad accumulation)")

print("\nStarting DPO training...")
dpo_results = dpo_trainer.train()

print("\nTraining completed!")
print(f"Final loss: {dpo_results.training_loss:.4f}")

print(f"Saving model to {output_dir}")
dpo_trainer.save_model(output_dir)

print("\nTraining metrics summary:")
for name, value in dpo_results.metrics.items():
    if isinstance(value, (int, float)):
        print(f"- {name}: {value:.4f}")


Starting DPO training:
- Training examples: 68
- Evaluation examples: 17
- Model: unsloth/Llama-3.2-3B-Instruct
- Output directory: outputs/symptom_dpo_20251210_192023
- Beta value: 0.1
- Training epochs: 1
- Learning rate: 1e-06
- Batch size: 2 * 8 (grad accumulation)

Starting DPO training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 68 | Num Epochs = 1 | Total steps = 5
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 4,587,520 of 3,217,337,344 (0.14% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss



Training completed!
Final loss: 0.0440
Saving model to outputs/symptom_dpo_20251210_192023

Training metrics summary:
- train_runtime: 18.7556
- train_samples_per_second: 3.6260
- train_steps_per_second: 0.2670
- total_flos: 0.0000
- train_loss: 0.0440
- epoch: 1.0000


## DPO Evaluation

In [ ]:
from unsloth import FastLanguageModel

FastLanguageModel.for_inference(model)

print("=== DPO MODEL EVALUATION (STRICT FORMAT) ===")

test_prompts = [
    "I feel strange and tired but nothing hurts.", # Happy Case 1 - Unknown
    "My eyes are itchy and my nose won’t stop running.", # Happy Case 2 - Allergies
    "I have a high fever, cough, and body aches.", # Happy Case 3 - Flu
    "My knee hurts when I jump", # Edge Case 1
    "Hi!", # Edge Case 2
    "My throat hurts, my eyes are itchy, and I have diarrhea" # Failure Case
]

for i, prompt in enumerate(test_prompts):
    print("\n" + "="*90)
    print(f"Prompt {i}: {prompt}\n")
    output = symptom_inference(prompt)
    print("Model Output:\n")
    print(output)
    print("="*90)



=== DPO MODEL EVALUATION (STRICT FORMAT) ===

Prompt 0: I feel strange and tired but nothing hurts.

Model Output:

Condition: Unknown
Guidance: I cannot confidently match these symptoms to any known condition. Please monitor your body and consider getting tested for unknown reasons. Rest and stay hydrated.

Prompt 1: My eyes are itchy and my nose won’t stop running.

Model Output:

Condition: Allergies
Guidance: It's possible that your symptoms are related to allergies. Try avoiding known triggers like pollen, dust, or certain foods, and take an antihistamine to see if it relieves your symptoms. Stay hydrated by drinking plenty of water. Safety note: This is not a diagnosis. Please consult a healthcare professional if symptoms worsen or persist.

Prompt 2: I have a high fever, cough, and body aches.

Model Output:

Condition: Flu
Guidance: Take care of yourself with plenty of rest, hydration, and warm fluids. Use over-the-counter medications to manage fever and cough. Safety note: Thi

## Save Model after DPO Training

In [ ]:
model.save_pretrained("symptom_sft_dpo")
tokenizer.save_pretrained("symptom_sft_dpo")

('symptom_sft_dpo/tokenizer_config.json',
 'symptom_sft_dpo/special_tokens_map.json',
 'symptom_sft_dpo/chat_template.jinja',
 'symptom_sft_dpo/tokenizer.json')

# Upload Model to Hugging Face for Easy Access

In [ ]:
# from huggingface_hub import login
# login()

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors: 100%|##########| 18.4MB / 18.4MB            

  ...om_sft_dpo/tokenizer.json: 100%|##########| 17.2MB / 17.2MB            

No files have been modified since last commit. Skipping to prevent empty commit.


...

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors: 100%|##########| 18.4MB / 18.4MB            

  ...-sft-final/tokenizer.json: 100%|##########| 17.2MB / 17.2MB            

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/ras6899/symptom-sft-base/commit/85944e03a50eb29a4aa0411620a18036428cb628', commit_message='Upload folder using huggingface_hub', commit_description='', oid='85944e03a50eb29a4aa0411620a18036428cb628', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ras6899/symptom-sft-base', endpoint='https://huggingface.co', repo_type='model', repo_id='ras6899/symptom-sft-base'), pr_revision=None, pr_num=None)

In [ ]:
# uncomment this code cell if you want to update the sft base or dpo model

"""
from huggingface_hub import upload_folder

upload_folder(
    folder_path="symptom_sft_dpo",
    repo_id="ras6899/symptom-sft-dpo",
    repo_type="model"
)
"""

"""
upload_folder(
    folder_path="model-medical-sft-final",
    repo_id="ras6899/symptom-sft-base",
    repo_type="model"
)
"""


# Gradio App for Front-End

In [ ]:
from unsloth import FastLanguageModel
import torch

dpo_model, dpo_tokenizer = FastLanguageModel.from_pretrained(
    "ras6899/symptom-sft-dpo",
    max_seq_length=2048,
    dtype=torch.bfloat16,
    load_in_4bit=False
)


FastLanguageModel.for_inference(dpo_model)

def generate(symptoms):
    prompt = symptoms
    inputs = dpo_tokenizer(prompt, return_tensors="pt").to("cuda")
    output = dpo_model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.3
    )
    return dpo_tokenizer.decode(output[0], skip_special_tokens=True)

print(generate("I have a sore throat and mild coughing."))


==((====))==  Unsloth 2025.12.4: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

unsloth/Llama-3.2-3B-Instruct does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.
I have a sore throat and mild coughing. I think I might have Covid-19. How can I take care of myself today?
Take care of your overall health with a balanced diet rich in vitamins and minerals. Drink plenty of warm fluids, such as water, tea, or soup, to help soothe your throat. Consider using a humidifier to add moisture to the air and reduce coughing. Rest and manage your stress levels through relaxation techniques like deep breathing or meditation. Avoid close contact with others to prevent transmission. Next steps: Monitor your symptoms and consult a healthcare professional if they worsen. Safety note: This is not a diagnosis. Please consult a healthcare professional if symptoms worsen or persist. ClassSymptomsClass: SymptomsClass SymptomClass “Class” Class “Class” Class SymptomClass SymptomClass “Class” Guidance Guidance Guidance Guidance Guidance Guidance Guidance Guidance 

In [ ]:
import gradio as gr
from unsloth import FastLanguageModel
import torch

# ================================
# Load your DPO model
# ================================
model, tokenizer = FastLanguageModel.from_pretrained(
    "ras6899/symptom-sft-dpo",
    max_seq_length=2048,
    load_in_4bit=True,
)

FastLanguageModel.for_inference(model)
model.eval()


# ================================
# Inference function (FINAL VERSION)
# ================================
def classify_symptoms(symptoms):
    system_prompt = (
        "Classify symptoms into EXACTLY ONE of the following conditions: "
        "cold, flu, COVID-19, allergies, strep throat, stomach flu, bronchitis, Unknown. "
        "Choose Unknown if symptoms do not clearly match a condition.\n\n"
        "Output format:\n"
        "Condition: <one condition>\n"
        "Guidance: <one short, calm paragraph with 2–3 gentle suggestions>\n"
        "Safety note: This is not a diagnosis. Please consult a healthcare professional if symptoms worsen or persist.\n"
    )

    # Correct Llama 3 formatting for Unsloth
    prompt = (
        f"<|begin_of_text|>"
        f"<|start_header_id|>system<|end_header_id|>\n"
        f"{system_prompt}<|eot_id|>"
        f"<|start_header_id|>user<|end_header_id|>\n"
        f"{symptoms}<|eot_id|>"
        f"<|start_header_id|>assistant<|end_header_id|>\n"
    )

    device = next(model.parameters()).device
    encoded = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        output = model.generate(
            **encoded,
            max_new_tokens=180,
            temperature=0.1,
            top_p=0.9,
            repetition_penalty=1.15,   # Prevents "NextSymptoms:" / repeats
        )

    decoded = tokenizer.decode(output[0], skip_special_tokens=False)

    # Extract assistant response cleanly
    if "<|start_header_id|>assistant<|end_header_id|>" in decoded:
        decoded = decoded.split("<|start_header_id|>assistant<|end_header_id|>")[-1]

    return decoded.replace("<|eot_id|>", "").strip()


# ================================
# UT Austin UI Layout
# ================================
import gradio as gr

ut_css = """
:root {
    --ut-orange: #BF5700;
    --ut-dark: #333333;
    --ut-light: #FFFFFF;
}

body {
    background: #F5F5F5 !important;
}

.gradio-container {
    font-family: 'Inter', sans-serif;
}

#title-bar {
    background: var(--ut-orange);
    color: white;
    padding: 22px;
    text-align: center;
    border-radius: 12px;
    font-size: 32px;
    font-weight: 700;
}

#sub-header {
    text-align: center;
    color: var(--ut-dark);
    font-size: 16px;
    margin-top: 6px;
    margin-bottom: 20px;
}

.input-card, .output-card {
    background: white;
    padding: 18px;
    border-radius: 14px;
    border: 2px solid var(--ut-orange);
    box-shadow: 0px 1px 4px rgba(0,0,0,0.08);
}
"""

with gr.Blocks(css=ut_css, theme=gr.themes.Soft()) as demo:
    gr.HTML("<div id='title-bar'>Campus Care AI 🤘</div>")
    gr.HTML("<div id='sub-header'>Powered by a Fine-Tuned Llama Model — Not a medical diagnosis</div>")

    with gr.Row():
        with gr.Column(scale=1):
            symptoms_input = gr.Textbox(
                label="Describe your symptoms",
                placeholder="e.g., I have a fever, chills, and body aches...",
                lines=4,
                elem_classes=["input-card"],
            )
            submit_btn = gr.Button("Analyze Symptoms", variant="primary")

        with gr.Column(scale=1):
            output_box = gr.Textbox(
                label="Condition & Guidance",
                lines=8,
                elem_classes=["output-card"],
            )

    submit_btn.click(fn=classify_symptoms, inputs=symptoms_input, outputs=output_box)

demo.launch()


==((====))==  Unsloth 2025.12.4: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
unsloth/llama-3.2-3b-instruct-unsloth-bnb-4bit does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


/tmp/ipython-input-1868620157.py:110: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(css=ut_css, theme=gr.themes.Soft()) as demo:
/tmp/ipython-input-1868620157.py:110: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=ut_css, theme=gr.themes.Soft()) as demo:


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ba87bc7fcf402208da.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
